In [14]:
import pandas as pd
import numpy as np
import datetime as dt

prices = pd.read_csv('/content/airbnb_price.csv')
xls = pd.ExcelFile('/content/airbnb_room_type.xlsx')
room_type = xls.parse()

reviews = pd.read_csv('/content/airbnb_last_review.csv')

In [15]:
prices.head().style.background_gradient()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [23]:
room_type.head(10).style.background_gradient()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt
5,5295,Beautiful 1br on Upper West Side,Entire home/apt
6,5441,Central Manhattan/near Broadway,Private room
7,5803,"Lovely Room 1, Garden, Best Area, Legal rental",Private room
8,6021,Wonderful Guest Bedroom in Manhattan for SINGLES,Private room
9,6848,Only 2 stops to Manhattan studio,entire home/apt


In [17]:
reviews.head().style.background_gradient()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


#Cleaning the price column

In [18]:
prices["price"] = prices['price'].str.replace(' dollars', "") # removing whitespaces

prices["price"] = pd.to_numeric(prices["price"])  #converting data to numeric

print(prices['price'].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


#Calculate average price

the three quaters of listing cost $175

but there are some outliers in prices as the max price is $7500 per night.

In [19]:
free_listing = prices["price"] == 0

prices = prices.loc[~free_listing]

avg_price = round(prices["price"]. mean(), 2)

print("Average price per night for AirBnb in NYC is ${}.". format(avg_price))


Average price per night for AirBnb in NYC is $141.82.


#Comparing Cost of Private Retail Market.

According to Zumper, a 1 bedroom apart in NYC costs on average $3,100 per month.

In [22]:
prices["price_per_month"] = prices["price"] * 365 /12

avergae_price_per_month = round(prices['price_per_month'].mean(), 2)

print("Airbnb monthly costs are ${} , while in the private market you would pay {}.".format(avergae_price_per_month, "$3,100.00"))

Airbnb monthly costs are $4313.61 , while in the private market you would pay $3,100.00.


#Cleaning the room type Column

let's dig deeper into room_type and breakdown the listing  into
1. Private room
2. private room
3. PRIVATE ROOM

we will clean the room_type and list the properties with one description.

In [26]:
room_type['room_type'] = room_type['room_type'].str.lower()

room_type['room_type'] = room_type['room_type'].astype("category")

room_frequency = room_type["room_type"].value_counts()

print(room_frequency)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


#what timeframe are we working with ?

Looks like there is fairly similar sized market opportunity for both private rooms (45% of listing) and entire home/apt (52%) on Airbnb platform in NYC.

let's see our earliest and last review on the properties.


In [28]:
reviews["last_review"] =pd.to_datetime(reviews['last_review'])

first_reviewed = reviews['last_review'].dt.date.min()

last_reviewed = reviews['last_review'].dt.date.max()

print("the earliest Airbnb review is {}, and the latest review is {}".format(first_reviewed, last_reviewed))

the earliest Airbnb review is 2019-01-01, and the latest review is 2019-07-09


#Joining the dataframes

Now we have all the required and clean information in our objects. we will combine the data we have extracted and join the tables together to get a single datafram.


In [29]:
room_and_prices = prices.merge(room_type, how="outer", on="listing_id")

airbnb_merged = room_type.merge(room_and_prices, how="outer", on="listing_id")

airbnb_merged.dropna(inplace=True)

print("There are {} are duplicates in the DataFrame.". format(airbnb_merged.duplicated().sum()))

There are 0 are duplicates in the DataFrame.


In [31]:
airbnb_merged.head(15).style.background_gradient()

,listing_id,description_x,room_type_x,price,nbhood_full,price_per_month,description_y,room_type_y
0,2595,Skylit Midtown Castle,entire home/apt,225.000000,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt
1,3831,Cozy Entire Floor of Brownstone,entire home/apt,89.000000,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,200.000000,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,Large Furnished Room Near B'way,private room,79.000000,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,150.000000,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt
5,5295,Beautiful 1br on Upper West Side,entire home/apt,135.000000,"Manhattan, Upper West Side",4106.250000,Beautiful 1br on Upper West Side,entire home/apt
6,5441,Central Manhattan/near Broadway,private room,85.000000,"Manhattan, Hell's Kitchen",2585.416667,Central Manhattan/near Broadway,private room
7,5803,"Lovely Room 1, Garden, Best Area, Legal rental",private room,89.000000,"Brooklyn, South Slope",2707.083333,"Lovely Room 1, Garden, Best Area, Legal rental",private room
8,6021,Wonderful Guest Bedroom in Manhattan for SINGLES,private room,85.000000,"Manhattan, Upper West Side",2585.416667,Wonderful Guest Bedroom in Manhattan for SINGLES,private room
9,6848,Only 2 stops to Manhattan studio,entire home/apt,140.000000,"Brooklyn, Williamsburg",4258.333333,Only 2 stops to Manhattan studio,entire home/apt


#Analyzing lisiting prices by NYC borough

we need to extract the information from string and store it in new column for analysis.


In [33]:
airbnb_merged['borough'] =airbnb_merged["nbhood_full"].str.partition(",")[0]

boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

boroughs = boroughs.round(2).sort_values("mean", ascending=False)

print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898963.0  184.03   149.0  10319
Brooklyn       1275435.0  122.02    95.0  10453
Queens          320765.0   92.81    70.0   3456
Staten Island    22974.0   86.04    71.0    267
Bronx            55231.0   79.24    65.0    697


#price range by borough

We will assign the following categories and price ranges:

label	price

Budget	$0-69

Average	$70-175

Expensive	$176-350

Extravagant	> $350


In [36]:
label_names = ["Budget", "Avergae", "Expensive", " Extravagent"]

ranges = [0, 69, 175, 350, np.inf]

airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)

prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()

print(prices_by_borough)

borough        price_range 
Bronx          Budget           381
               Avergae          286
               Expensive         25
                Extravagent       5
Brooklyn       Budget          3194
               Avergae         5534
               Expensive       1466
                Extravagent     259
Manhattan      Budget          1148
               Avergae         5288
               Expensive       3073
                Extravagent     810
Queens         Budget          1632
               Avergae         1505
               Expensive        291
                Extravagent      28
Staten Island  Budget           124
               Avergae          123
               Expensive         20
                Extravagent       0
Name: price_range, dtype: int64


#Restults and Findings

1. Price Dataset : The average price per night for an Airbnb monthly cost is $141.82 , Airbnb monthly cost are $4313.61, while in the private market you would pay $3,100.00 ,
2. Room Type Dataset :  both private rooms (45% of listings) and entire homes/apartments (52%) on the Airbnb platform in NYC.
3. Review Dataset : The earliest Airbnb review is 2019-01-01, the latest review is 2019-07-09 which gives us a insight of property updates.
4. price Range : we mreged extracted inforamtion to find out how many properties are in budget, average, expensive & extravagant.
